To get started, enable [automatic reloading of modules] (https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload). This will allow you to modify the source code and immediately observe the effects of the changes in the notebook, without the need for reinitiating the kernel or reinstalling the package.

In [ ]:
%load_ext autoreload
%autoreload 2

Pre-load all the Qiskit Metal libraries that are needed for the rest of this notebook.

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

from qiskit_metal.renderers.renderer_gds.gds_renderer import QGDSRenderer

### Confirm QDesign is able to load your renderer
Create a QDesign instance.

In [ ]:
design = designs.DesignPlanar()

Ensure the EMPro renderer is initiated.

In [ ]:
design.renderers.keys()

For convenience, let's create a short-handle alias to refer to the renderer during the remainder of this notebook.

In [ ]:
a_empro = design.renderers.empro

Play around with the renderer by modifying the default options.

In [ ]:
a_empro.options.height = '30.0'
a_empro.options

Original values will continue being accessible like so:

In [ ]:
a_empro.get_template_options(design)

### Populate a sample QDesign to demonstrate interaction with the renderer

In [ ]:
gui = MetalGUI(design)
design.overwrite_enabled = True

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [ ]:
## Custom options for all the transmons
options = dict(
    pad_width = '425 um',
    pad_gap = '80 um',
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict( 
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 TransmonPockets
q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_02', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.9mm', orientation = '90', gds_cell_name='FakeJunction_02', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_01',**options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.9mm', orientation = '90', gds_cell_name='my_other_junction', **options))

options = Dict(
    meander=Dict(
        lead_start='0.1mm',
        lead_end='0.1mm',
        asymmetry='0 um')
)

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='50um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight='0.13mm',
            end_straight='0.13mm'
        ),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 90
cpw1 = connect('cpw1', 'Q1', 'd', 'Q2', 'c', '5.7 mm', f'+{asym}um', fillet='25um')
cpw2 = connect('cpw2', 'Q3', 'c', 'Q2', 'a', '5.6 mm', f'-{asym}um', flip=True, fillet='100um')
cpw3 = connect('cpw3', 'Q3', 'a', 'Q4', 'b', '5.5 mm', f'+{asym}um', fillet='75um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'd', '5.8 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

### Export list of the design QGeometries to file using QEMProRenderer
The QEMProRenderer class contains several methods. Let's use one intended to print out the name of the QGeometry tables to a text file (Remember: QGeometry contains the list of the raw layout shapes that compose the design, which we have created in the previous cell).

In [ ]:
a_empro.write_qgeometry_table_names_to_file('./simple_output.txt')

Here another example where we sub select a single QComponent instance (`cpw1`) of type RouteMeander.  This will only export the name of tables containing shapes related to that instance, which in this case is only paths, and not junctions or poly.

In [ ]:
a_empro.write_qgeometry_table_names_to_file('./simple_output_cpw1.txt',highlight_qcomponents=['cpw1'])

### Adding columns to the renderer

In [ ]:
design.qgeometry.tables['junction']

In [ ]:
q1.options.empro_a_column_name = 'q1 empro'
q2.options.empro_a_column_name = 'q2 empro'
q3.options.empro_a_column_name = 'q3 empro'
q4.options.empro_a_column_name = 'q4 empro'

gui.rebuild()
gui.autoscale()

design.qgeometry.tables['junction']

We can also delete components

In [ ]:
q1.delete()
q2.delete()
q3.delete()
q4.delete()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_02', empro_a_column_name='q1 empro 2', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.9mm', orientation = '90', gds_cell_name='FakeJunction_02', empro_a_column_name='q2 empro 2', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_01', empro_a_column_name='q3 empro 2', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.9mm', orientation = '90', gds_cell_name='my_other_junction', empro_a_column_name='q4 empro 2', **options))

design.qgeometry.tables['junction']

### Accessing information and methods

In [ ]:
a_empro.design.components.keys()

In [ ]:
a_empro.design.qgeometry.tables.keys()

In [ ]:
a_empro.design.renderers.keys()

The base QRenderer class comes with useful methods to more easily access some of the information. You will find more method described in the QRenderer documentation. The example below for example returns the QComponent's IDs.

In [ ]:
a_empro.get_unique_component_ids(highlight_qcomponents = ['Q1', 'Q1', 'Q4', 'cpw1', 'cpw2', 'cpw3', 'cpw4'])

The following instead shows three ways to access the same QGeometry table.

In [ ]:
a_empro.design.components['Q1'].qgeometry_table('junction') # via QComonent name
a_empro.design._components[9].qgeometry_table('junction') # via QComponent ID
q1.qgeometry_table('junction') # via the QComponent instance

The method `QEMProRenderer.get_qgeometry_tables_for_empro()` exemplifies how to iterate through chips and tables.

In [ ]:
from qiskit_metal.renderers.renderer_keysight.empro_renderer import QEMProRenderer
?QEMProRenderer.get_qgeometry_tables_for_empro

### Communicate state
We can also interact with any other method of the QDesign instance, for example we can generate a warning into the logger as shown in the next cell.  This is particularly useful to document problems with the user defined QRenderer execution

In [ ]:
# Purposefully generates a warning message.  
a_empro.logger.warning('Show a warning message for plugin developer.')

## Qiskit Metal Version

In [ ]:
metal.about();

In [ ]:
# This command below is if the user wants to close the Metal GUI.
gui.main_window.close()